### Obtain the result table: For the added edges of EWCSN, its weight is the average of the existing edge weights.

#### Generate the true.csv and pred.csv files for all the results of this experiment.

##### Mouse

In [ ]:
import os
import numpy as np
import pandas as pd

datasets = ['Baron_Mouse', 'AMB', 'TM']
base_path = "../../result/ewcsn_preds/"
save_folder = "../Tables"
net_files = ["BIOGRID_mouse", "AGR_MGI"]
for dataset in datasets:
    print("processing dataset: ", dataset)
    for net_name in net_files:
        print("net name: ", net_name)
        save_data_folder = os.path.join(save_folder, dataset)
        os.makedirs(save_data_folder, exist_ok=True)
        seq_dict_file = f'../../result/datasets/{dataset}/seq_dict.npz'
        seq_dict = np.load(seq_dict_file, allow_pickle=True) 
        str_labels = seq_dict['str_labels']

        data_file = os.path.join(base_path, f"{dataset}_{net_name}_a0.01_hvgs2000_prediction.h5")
        cell_type = pd.read_hdf(data_file, key='cell_type')
        true_labels = cell_type['true_cell_type'].values
        pred_labels = cell_type['pred_cell_type'].values
        true_str_labels = [str_labels[label] for label in true_labels]
        pred_str_labels = [str_labels[label] for label in pred_labels]


        true_df = pd.DataFrame(true_str_labels)
        pred_df = pd.DataFrame(pred_str_labels)
        
        true_label_csv_path = os.path.join(save_data_folder, f"ewcsn_{net_name}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"ewcsn_{net_name}_pred.csv")

        true_df.to_csv(true_label_csv_path, index=False)
        pred_df.to_csv(pred_label_csv_path, index=False)   


        print(f"Saved true labels to {true_label_csv_path}")
        print(f"Saved predicted labels to {pred_label_csv_path}")
        print("\n")
    
    print("\n")

##### Human

In [ ]:
import os
import numpy as np
import pandas as pd

datasets = ['Baron_Human', 'Zhang_T', 'Kang_ctrl','Zheng68K',"Muraro"]
base_path = "../../result/ewcsn_preds/"
save_folder = "../Tables"
net_files = ["BIOGRID_human", "AGR_human", "HumanNet-GSP"]
for dataset in datasets:
    print("processing dataset: ", dataset)
    for net_name in net_files:
        print("net name: ", net_name)
        save_data_folder = os.path.join(save_folder, dataset)
        os.makedirs(save_data_folder, exist_ok=True)
        seq_dict_file = f'../../result/datasets/{dataset}/seq_dict.npz'
        seq_dict = np.load(seq_dict_file, allow_pickle=True) 
        str_labels = seq_dict['str_labels']

        data_file = os.path.join(base_path, f"{dataset}_{net_name}_a0.01_hvgs2000_prediction.h5")
        cell_type = pd.read_hdf(data_file, key='cell_type')

        true_labels = cell_type['true_cell_type'].values
        pred_labels = cell_type['pred_cell_type'].values

        true_str_labels = [str_labels[label] for label in true_labels]
        pred_str_labels = [str_labels[label] for label in pred_labels]


        true_df = pd.DataFrame(true_str_labels)
        pred_df = pd.DataFrame(pred_str_labels)
        
        true_label_csv_path = os.path.join(save_data_folder, f"ewcsn_{net_name}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"ewcsn_{net_name}_pred.csv")

        true_df.to_csv(true_label_csv_path, index=False)
        pred_df.to_csv(pred_label_csv_path, index=False)   


        print(f"Saved true labels to {true_label_csv_path}")
        print(f"Saved predicted labels to {pred_label_csv_path}")
        print("\n")
    
    print("\n")

#### Based on the true.csv and pred.csv files, calculate accuracy (acc) and mean F1 score, and generate a summary table.

##### Mouse

In [ ]:
from sklearn.metrics import precision_score,f1_score, accuracy_score
import os
import numpy as np
import pandas as pd

datasets = [ 'AMB', 'TM', 'Baron_Mouse']
dataset_names = ['AMB', 'TM', 'Baron Mouse']
methods = ["wcsgnet_Log", "ewcsn_BIOGRID_mouse", "ewcsn_AGR_MGI"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Mouse)", "EWCSGNet(AGR_Mouse)"]

save_folder = "../Tables"

mean_f1_results = pd.DataFrame(index=method_names, columns=dataset_names)
acc_results = pd.DataFrame(index=method_names, columns=dataset_names)

for dataset, dataset_name in zip(datasets, dataset_names):
    print("processing dataset: ", dataset)
    save_data_folder = os.path.join(save_folder, dataset)

    f1_results = pd.DataFrame()

    for method, method_name in zip(methods, method_names):
        print("method: ", method_name)
        true_label_csv_path = os.path.join(save_data_folder, f"{method}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"{method}_pred.csv")    
        true_df = pd.read_csv(true_label_csv_path)
        pred_df = pd.read_csv(pred_label_csv_path)       

        label_true = true_df.iloc[:, 0].values 
        label_pred = pred_df.iloc[:, 0].values    
        unique_classes = np.unique(label_true)  

        test_acc = accuracy_score(label_true, label_pred)
        test_f1 = f1_score(label_true, label_pred, average='macro', labels=unique_classes)

        test_f1_all = f1_score(label_true, label_pred, average=None, labels=unique_classes)      
        test_acc = accuracy_score(label_true, label_pred)
        test_f1 = f1_score(label_true, label_pred, average='macro', labels=unique_classes)
        mean_f1_results.loc[method_name, dataset_name] = test_f1
        acc_results.loc[method_name, dataset_name] = test_acc 
        print('Acc: %.03f, Mean-F1: %.03f'%(test_acc, test_f1))  

os.makedirs(os.path.join(save_folder, "ewcsn_F1_Acc"), exist_ok=True)
f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", f"Mouse_Mean_F1.csv")
mean_f1_results.to_csv(f1_path)
print(f"Mean-F1 scores saved to {f1_path}")

acc_path = os.path.join(save_folder, "ewcsn_F1_Acc", f"Mouse_Acc.csv")
acc_results.to_csv(acc_path)
print(f"Accuracy saved to {acc_path}")

##### Human

In [ ]:
from sklearn.metrics import precision_score,f1_score, accuracy_score
import os
import numpy as np
import pandas as pd

datasets = ['Zhang_T', 'Kang_ctrl', 'Zheng68K','Baron_Human', 'Muraro']
dataset_names = ['Zhang T', 'Kang', 'Zheng 68k', 'Baron Human', "Muraro"]
methods = ["wcsgnet_Log", "ewcsn_BIOGRID_human", "ewcsn_AGR_human", "ewcsn_HumanNet-GSP"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Human)", "EWCSGNet(AGR_Human)", "EWCSGNet(HumanNet-GSP)"]

save_folder = "../Tables"

mean_f1_results = pd.DataFrame(index=method_names, columns=dataset_names)
acc_results = pd.DataFrame(index=method_names, columns=dataset_names)

for dataset, dataset_name in zip(datasets, dataset_names):
    print("processing dataset: ", dataset)
    save_data_folder = os.path.join(save_folder, dataset)
    f1_results = pd.DataFrame()

    for method, method_name in zip(methods, method_names):
        print("method: ", method_name)
        true_label_csv_path = os.path.join(save_data_folder, f"{method}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"{method}_pred.csv")    
        true_df = pd.read_csv(true_label_csv_path)
        pred_df = pd.read_csv(pred_label_csv_path)       

        label_true = true_df.iloc[:, 0].values
        label_pred = pred_df.iloc[:, 0].values
        
        unique_classes = np.unique(label_true)  
        test_acc = accuracy_score(label_true, label_pred)
        test_f1 = f1_score(label_true, label_pred, average='macro', labels=unique_classes)

        test_f1_all = f1_score(label_true, label_pred, average=None, labels=unique_classes)       
        test_acc = accuracy_score(label_true, label_pred)
        test_f1 = f1_score(label_true, label_pred, average='macro', labels=unique_classes)
        mean_f1_results.loc[method_name, dataset_name] = test_f1
        acc_results.loc[method_name, dataset_name] = test_acc 
        print('Acc: %.03f, Mean-F1: %.03f'%(test_acc, test_f1))  

os.makedirs(os.path.join(save_folder, "ewcsn_F1_Acc"), exist_ok=True)
f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", f"Human_Mean_F1.csv")
mean_f1_results.to_csv(f1_path)
print(f"Mean-F1 scores saved to {f1_path}")

acc_path = os.path.join(save_folder, "ewcsn_F1_Acc", f"Human_Acc.csv")
acc_results.to_csv(acc_path)
print(f"Accuracy saved to {acc_path}")

##### rare cell type

In [ ]:
from sklearn.metrics import precision_score,f1_score, accuracy_score
import os
import numpy as np
import pandas as pd

datasets = [ 'AMB', 'TM', 'Baron_Mouse']
dataset_names = ['AMB', 'TM', 'Baron Mouse']
methods = ["wcsgnet_Log", "ewcsn_BIOGRID_mouse", "ewcsn_AGR_MGI"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Mouse)", "EWCSGNet(AGR_Mouse)"]

save_folder = "../Tables"

rare_f1_results = pd.DataFrame(index=method_names, columns=dataset_names)

for dataset, dataset_name in zip(datasets, dataset_names):
    print("processing dataset: ", dataset)
    save_data_folder = os.path.join(save_folder, dataset)
    f1_results = pd.DataFrame()

    for method, method_name in zip(methods, method_names):
        print("method: ", method_name)
        true_label_csv_path = os.path.join(save_data_folder, f"{method}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"{method}_pred.csv")    
        true_df = pd.read_csv(true_label_csv_path)
        pred_df = pd.read_csv(pred_label_csv_path)       

        label_true = true_df.iloc[:, 0].values  
        label_pred = pred_df.iloc[:, 0].values
        unique, counts = np.unique(label_true, return_counts=True)
        total_cells = len(label_true)
        cell_frequencies = dict(zip(unique, counts))

        rare_classes = [cell_type for cell_type, count in cell_frequencies.items() if count / total_cells < 0.03]

        if not rare_classes:
            print(f"No rare cell types in dataset {dataset_name} for method {method_name}")
            rare_f1_results.loc[method_name, dataset_name] = np.nan
            continue

        rare_f1 = f1_score(label_true, label_pred, average='macro', labels=rare_classes)
        rare_f1_results.loc[method_name, dataset_name] = rare_f1
        print(f"Rare cell type F1 score for {method_name} in {dataset_name}: {rare_f1}")

os.makedirs(os.path.join(save_folder, "ewcsn_F1_Acc"), exist_ok=True)

rare_cell_type_f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", "mouse_rare_cell_type_mean-F1.csv")
rare_f1_results.to_csv(rare_cell_type_f1_path)
print(f"Rare-F1 scores saved to {rare_cell_type_f1_path}")

In [ ]:
from sklearn.metrics import precision_score,f1_score, accuracy_score
import os
import numpy as np
import pandas as pd

datasets = ['Zhang_T', 'Kang_ctrl', 'Zheng68K','Baron_Human', 'Muraro']
dataset_names = ['Zhang T', 'Kang', 'Zheng 68k', 'Baron Human', "Muraro"]
methods = ["wcsgnet_Log", "ewcsn_BIOGRID_human", "ewcsn_AGR_human", "ewcsn_HumanNet-GSP"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Human)", "EWCSGNet(AGR_Human)", "EWCSGNet(HumanNet-GSP)"]

save_folder = "../Tables"

rare_f1_results = pd.DataFrame(index=method_names, columns=dataset_names)

for dataset, dataset_name in zip(datasets, dataset_names):
    print("processing dataset: ", dataset)
    save_data_folder = os.path.join(save_folder, dataset)

    f1_results = pd.DataFrame()

    for method, method_name in zip(methods, method_names):
        print("method: ", method_name)
        true_label_csv_path = os.path.join(save_data_folder, f"{method}_true.csv")
        pred_label_csv_path = os.path.join(save_data_folder, f"{method}_pred.csv")    
        true_df = pd.read_csv(true_label_csv_path)
        pred_df = pd.read_csv(pred_label_csv_path)       

        label_true = true_df.iloc[:, 0].values 
        label_pred = pred_df.iloc[:, 0].values
        unique, counts = np.unique(label_true, return_counts=True)
        total_cells = len(label_true)
        cell_frequencies = dict(zip(unique, counts))

        rare_classes = [cell_type for cell_type, count in cell_frequencies.items() if count / total_cells < 0.03]

        if not rare_classes:
            print(f"No rare cell types in dataset {dataset_name} for method {method_name}")
            rare_f1_results.loc[method_name, dataset_name] = np.nan
            continue

        rare_f1 = f1_score(label_true, label_pred, average='macro', labels=rare_classes)
        rare_f1_results.loc[method_name, dataset_name] = rare_f1
        print(f"Rare cell type F1 score for {method_name} in {dataset_name}: {rare_f1}")

os.makedirs(os.path.join(save_folder, "ewcsn_F1_Acc"), exist_ok=True)

rare_cell_type_f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", "human_rare_cell_type_mean-F1.csv")
rare_f1_results.to_csv(rare_cell_type_f1_path)
print(f"Rare-F1 scores saved to {rare_cell_type_f1_path}")

### Plot

#### Mouse

##### bar: Acc 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
acc_path = os.path.join(save_folder, "ewcsn_F1_Acc", "Mouse_Acc.csv")
acc_results = pd.read_csv(acc_path, index_col=0)

dataset_names = ['AMB', 'TM', 'Baron Mouse']
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Mouse)", "EWCSGNet(AGR_Mouse)"]

acc_results = acc_results.loc[method_names]

WCSGNet = acc_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_Mouse = acc_results.loc["EWCSGNet(BioGRID_Mouse)", dataset_names]
AGR_Mouse = acc_results.loc["EWCSGNet(AGR_Mouse)", dataset_names]

x = np.arange(len(dataset_names)) 
width = 0.23  

plt.figure(figsize=(5, 3.5))

plt.bar(x - width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x, BioGRID_Mouse, width, label='EWCSGNet(BioGRID_Mouse)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + width, AGR_Mouse, width, label='EWCSGNet(AGR_Mouse)', color='#95A3C3', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=0)  
plt.ylim(0.8, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)  

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}', 
                    xy=(bar.get_x() + bar.get_width() / 2, height),  
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, 
                    rotation=90) 

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/mouse_ewcsn_acc.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/mouse_ewcsn_acc.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()


##### bar: mean F1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", "Mouse_Mean_F1.csv")
f1_results = pd.read_csv(f1_path, index_col=0)

dataset_names = ['AMB', 'TM', 'Baron Mouse']
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Mouse)", "EWCSGNet(AGR_Mouse)"]

f1_results = f1_results.loc[method_names]

WCSGNet = f1_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_Mouse = f1_results.loc["EWCSGNet(BioGRID_Mouse)", dataset_names]
AGR_Mouse = f1_results.loc["EWCSGNet(AGR_Mouse)", dataset_names]

x = np.arange(len(dataset_names)) 
width = 0.23 

plt.figure(figsize=(5, 3.5))

plt.bar(x - width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x, BioGRID_Mouse, width, label='EWCSGNet(BioGRID_Mouse)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + width, AGR_Mouse, width, label='EWCSGNet(AGR_Mouse)', color='#95A3C3', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=0) 
plt.ylim(0.8, 1.0) 

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)  

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, 
                    rotation=90)  

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/mouse_ewcsn_mean_F1.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/mouse_ewcsn_mean_F1.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()


##### rare cell type: F1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", "mouse_rare_cell_type_mean-F1.csv")
f1_results = pd.read_csv(f1_path, index_col=0)

dataset_names = ['AMB', 'TM', 'Baron Mouse']
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Mouse)", "EWCSGNet(AGR_Mouse)"]

f1_results = f1_results.loc[method_names]

WCSGNet = f1_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_Mouse = f1_results.loc["EWCSGNet(BioGRID_Mouse)", dataset_names]
AGR_Mouse = f1_results.loc["EWCSGNet(AGR_Mouse)", dataset_names]

x = np.arange(len(dataset_names)) 
width = 0.23  

plt.figure(figsize=(5, 3.5))

plt.bar(x - width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x, BioGRID_Mouse, width, label='EWCSGNet(BioGRID_Mouse)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + width, AGR_Mouse, width, label='EWCSGNet(AGR_Mouse)', color='#95A3C3', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=0)  
plt.ylim(0.8, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False) 

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), 
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8,  
                    rotation=90)  

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/mouse_rare_ewcsn_mean_F1.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/mouse_rare_ewcsn_mean_F1.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()

#### Human

##### bar: Acc

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
acc_path = os.path.join(save_folder, "ewcsn_F1_Acc", "Human_Acc.csv")
acc_results = pd.read_csv(acc_path, index_col=0)

dataset_names = ['Zhang T', 'Kang', 'Zheng 68k', 'Baron Human', "Muraro"] 
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Human)", "EWCSGNet(AGR_Human)", "EWCSGNet(HumanNet-GSP)"]

acc_results = acc_results.loc[method_names]

WCSGNet = acc_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_human = acc_results.loc["EWCSGNet(BioGRID_Human)", dataset_names]
AGR_human = acc_results.loc["EWCSGNet(AGR_Human)", dataset_names]
HumanNet_GSP = acc_results.loc["EWCSGNet(HumanNet-GSP)", dataset_names]

x = np.arange(len(dataset_names))  
width = 0.23  

plt.figure(figsize=(5, 4))

plt.bar(x - 1.5 * width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x - 0.5 * width, BioGRID_human, width, label='EWCSGNet(BioGRID_Human)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + 0.5 * width, AGR_human, width, label='EWCSGNet(AGR_Human)', color='#95A3C3', edgecolor='white', linewidth=1)
plt.bar(x + 1.5 * width, HumanNet_GSP, width, label='EWCSGNet(HumanNet-GSP)', color='#F3C678', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=45)  
plt.ylim(0.5, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False) 

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height), 
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, 
                    rotation=90) 

add_value_labels(ax)
plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/human_ewcsn_acc.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/human_ewcsn_acc.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()


##### bar: mean-F1

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
acc_path = os.path.join(save_folder, "ewcsn_F1_Acc", "Human_Mean_F1.csv")
acc_results = pd.read_csv(acc_path, index_col=0)

dataset_names = ['Zhang T', 'Kang', 'Zheng 68k', 'Baron Human', "Muraro"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Human)", "EWCSGNet(AGR_Human)", "EWCSGNet(HumanNet-GSP)"]


acc_results = acc_results.loc[method_names]

WCSGNet = acc_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_human = acc_results.loc["EWCSGNet(BioGRID_Human)", dataset_names]
AGR_human = acc_results.loc["EWCSGNet(AGR_Human)", dataset_names]
HumanNet_GSP = acc_results.loc["EWCSGNet(HumanNet-GSP)", dataset_names]

x = np.arange(len(dataset_names)) 
width = 0.23  

plt.figure(figsize=(5, 4))

plt.bar(x - 1.5 * width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x - 0.5 * width, BioGRID_human, width, label='EWCSGNet(BioGRID_Human)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + 0.5 * width, AGR_human, width, label='EWCSGNet(AGR_Human)', color='#95A3C3', edgecolor='white', linewidth=1)
plt.bar(x + 1.5 * width, HumanNet_GSP, width, label='EWCSGNet(HumanNet-GSP)', color='#F3C678', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=45) 
plt.ylim(0.5, 1.0)  

ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)  
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height),  
                    xytext=(0, 3),  
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, 
                    rotation=90)  

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/human_ewcsn_mean_F1.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/human_ewcsn_mean_F1.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()


##### bar: rare cell type

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['font.family'] = 'Times New Roman'

save_folder = "../Tables"
f1_path = os.path.join(save_folder, "ewcsn_F1_Acc", "human_rare_cell_type_mean-F1.csv")
f1_results = pd.read_csv(f1_path, index_col=0)


dataset_names = ['Zhang T', 'Kang', 'Zheng 68k', 'Baron Human', "Muraro"]
method_names = ["WCSGNet(Logarithmic Transformation)", "EWCSGNet(BioGRID_Human)", "EWCSGNet(AGR_Human)", "EWCSGNet(HumanNet-GSP)"]

f1_results = f1_results.loc[method_names]

WCSGNet = f1_results.loc["WCSGNet(Logarithmic Transformation)", dataset_names]
BioGRID_human = f1_results.loc["EWCSGNet(BioGRID_Human)", dataset_names]
AGR_human = f1_results.loc["EWCSGNet(AGR_Human)", dataset_names]
HumanNet_GSP = f1_results.loc["EWCSGNet(HumanNet-GSP)", dataset_names]

x = np.arange(len(dataset_names))  
width = 0.23  

plt.figure(figsize=(5, 4))

plt.bar(x - 1.5 * width, WCSGNet, width, label='WCSGNet(Logarithmic Transformation)', color='#72B6A1', edgecolor='white', linewidth=1)
plt.bar(x - 0.5 * width, BioGRID_human, width, label='EWCSGNet(BioGRID_Human)', color='#E99675', edgecolor='white', linewidth=1)
plt.bar(x + 0.5 * width, AGR_human, width, label='EWCSGNet(AGR_Human)', color='#95A3C3', edgecolor='white', linewidth=1)
plt.bar(x + 1.5 * width, HumanNet_GSP, width, label='EWCSGNet(HumanNet-GSP)', color='#F3C678', edgecolor='white', linewidth=1)

plt.xticks(x, dataset_names, fontsize=10, rotation=45)  
plt.ylim(0.5, 1.0)  
ax = plt.gca()
ax.spines['top'].set_visible(False)  
ax.spines['right'].set_visible(False)  

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.4), fontsize=10, ncol=2, frameon=False)

def add_value_labels(ax):
    for bar in ax.patches:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',  
                    xy=(bar.get_x() + bar.get_width() / 2, height), 
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=8, 
                    rotation=90) 

add_value_labels(ax)

plt.tight_layout()

plt.savefig('../../result/Figures/ewcsn/human_rare_ewcsn_mean_F1.png', format='png', dpi=1200, bbox_inches='tight')
plt.savefig('../../result/Figures/ewcsn/human_rare_ewcsn_mean_F1.svg', format='svg', dpi=1200, bbox_inches='tight')

plt.show()
